In [2]:
%pip install --upgrade bigframes --progress-bar pretty

Note: you may need to restart the kernel to use updated packages.



Usage:   
  c:\Users\b0220\Documents\ReDrugAI\.venv\Scripts\python.exe -m pip install [options] <requirement specifier> [package-index-options] ...
  c:\Users\b0220\Documents\ReDrugAI\.venv\Scripts\python.exe -m pip install [options] -r <requirements file> [package-index-options] ...
  c:\Users\b0220\Documents\ReDrugAI\.venv\Scripts\python.exe -m pip install [options] [-e] <vcs project url> ...
  c:\Users\b0220\Documents\ReDrugAI\.venv\Scripts\python.exe -m pip install [options] [-e] <local project path> ...
  c:\Users\b0220\Documents\ReDrugAI\.venv\Scripts\python.exe -m pip install [options] <archive url/path> ...

option --progress-bar: invalid choice: 'pretty' (choose from 'auto', 'on', 'off', 'raw')


In [6]:
import bigframes.pandas as bpd

# Set your Google Cloud project ID.
# If you are running this in a Google Cloud environment, the project ID is often
# automatically detected.
bpd.options.bigquery.project = "redrugai"
bpd.options.bigquery.location = "US"

# Create a SQL query to select the first 50 rows with only the specified columns.
query = """
SELECT id, name, description, synonyms
FROM `bigquery-public-data.open_targets_platform.disease`
LIMIT 50
"""

# Load data from BigQuery using the query.
df = bpd.read_gbq(query)

# Display the DataFrame.
print(df.head())

             id                   name  \
0  DOID_0050890        synucleinopathy   
1    DOID_10113        trypanosomiasis   
2    DOID_10718             giardiasis   
3    DOID_13406  pulmonary sarcoidosis   
4     DOID_1947         trichomoniasis   

                                         description  \
0  A neurodegenerative disease that is characteri...   
1  Infection with protozoa of the genus trypanosoma.   
2  An infection of the small intestine caused by ...   
3  Sarcoidosis affecting the lung parenchyma. It ...   
4        An infection that is caused by Trichomonas.   

                                            synonyms  
0  {'hasExactSynonym': {'list': array([{'element'...  
1  {'hasExactSynonym': {'list': array([{'element'...  
2  {'hasExactSynonym': {'list': array([{'element'...  
3  {'hasExactSynonym': {'list': array([{'element'...  
4  {'hasExactSynonym': {'list': array([{'element'...  

[5 rows x 4 columns]


In [5]:

# Create a SQL query to count the total number of rows in the table.
count_query = "SELECT COUNT(*) as total_rows FROM `bigquery-public-data.open_targets_platform.disease`"

# Load the result from BigQuery using the query.
count_df = bpd.read_gbq(count_query)

# Display the total number of rows.
print("Total number of rows:", count_df['total_rows'][0])


Total number of rows: 38959
